# Upload to MongoDB

## Prerequisite
- Make sure you have already set up a MongoDB database on mongodb.com.
- You need to get the following informations and save them into a .env files: username, password, and server_address.

### Example .env File
username = wow680

password = rubbiSh

server_address = @niceMongoCluster.rrr.mongodb.net/

In [1]:
import random
import string
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.collation import Collation
from dotenv import load_dotenv
from datetime import datetime
import os
import json
import re

In [2]:
# CAUTION:
#   DO NOT MANUALLY USE THE FOLLOWING FUNCTIONS.
#   THEY ARE MEANT TO BE CALLED BY OTHER FUNCTIONS.
#   THAT WOULD APPROPRIATELY CLOSE THE CONNECTION USE.
#   SEE upload_to_mongo() FOR EXAMPLE.

def load_user_password():
    """
    Load the username and password from the .env file
    """
    if not os.path.isfile('.env'):
        print("\n\nError: No .env file found in the repository.\n\n")
        return None, None

    load_dotenv()
    return os.getenv('username'), os.getenv('password'), os.getenv('server_address')

def connect_to_mongo(username = None, password = None, server_address = None, debug = False):
    """
    DESCRIPTION:
        Return the MongoClient object
        Will load username, password, and server_address from the .env file
            if not provided as arguments.

    INPUT SIGNATURE:
        username: MongoDB username (string)
        password: MongoDB password (string)
        server_address: MongoDB server address (string)

    OUTPUT SIGNATURE:
        client: MongoClient object

    CAUTION:
        Manually setting username, password, and server_address
        when calling this function is highly discouraged. They are intended
        only to be used for quick connection to different MongoDB deployments
        when testing the application. The recommended way is to modify them
        from the .env file that should be in the same directory as this file.
    """

    if (username is None) or (password is None) or (server_address is None):
        username, password, server_address = load_user_password()
        uri = "mongodb+srv://" + username + ":" + password + server_address

    uri = "mongodb+srv://" + username + ":" + password + server_address
    client = MongoClient(uri, server_api=ServerApi('1'))

    # Send a ping to confirm a successful connection
    try:
        client.admin.command('ping')
        if debug:
            print("Pinged your deployment. You successfully connected to MongoDB!")
    except Exception as e:
        print("UNABLE TO CONNECT TO DATABASE")

    return client

In [3]:
def upload_to_mongoDB(database_name, collection_name, document):
    """
    DESCRIPTION:
        Upload a document to the MongoDB collection.
    
    INPUT SIGNATURE:
        database_name: MongoDB database name (string)
        collection_name: MongoDB collection name (string)
        document: the document to be uploaded (dictionary)

    OUTPUT SIGNATURE:
        The function will prints out debug messages if the upload is successful or not.
            This message is generated through the upload_to_mongoDB_helper function.
    """

    client = connect_to_mongo()
    db = client[database_name]
    collection = db[collection_name]

    # loop through every field of the document
    # if there is a field that is an empty string, replace it with None
    for key, value in document.items():
        if value == "":
            document[key] = None

    if collection_name in db.list_collection_names(): # if collection exists
        upload_to_mongoDB_helper(client, collection, document)
        
    else:
        upload_to_mongoDB_helper(client, collection, document)

def upload_to_mongoDB_helper(client_object, collection_object, document):
    """
    CAUTION:
        DO NOT USE THIS FUNCTION DIRECTLY. USE upload_to_mongoDB INSTEAD.
        
    DESCRIPTION:
        Upload a document to the MongoDB collection.
    """

    try:
        # Upload the document to the MongoDB collection
        result = collection_object.insert_one(document)

        # Print the inserted document's ID
        print(f"Document uploaded successfully. ID: {result.inserted_id}")
        client_object.close()
        return True

    except Exception as e:
        print(f"Error uploading document: {e}")
        client_object.close()
        return False

In [10]:
# config destination to upload
database_name = "initial_testing"
collection_name = "dummy_collection_1"
document = {
    'name' : 'Ann',
    'state' : 'NY'
}

In [11]:
# upload a document to the database
# if the collection or database does not exist, it will be created
upload_to_mongoDB(database_name, collection_name, document)

Document uploaded successfully. ID: 66713414d6ab98cc43b821dd
